In [106]:
import pandas as pd
import numpy as np

In [107]:
train_data_station1   = pd.read_csv('./train_station1.csv')
train_data_station2   = pd.read_csv('./train_Station2.csv')

test_data_station1   = pd.read_csv('./test_station1.csv')
test_data_station2   = pd.read_csv('./test_Station2.csv')

weather_data = pd.read_csv('./assets/weather_data_daytime.csv')

In [108]:
weather_data.shape

(2944, 25)

In [109]:
#maybe impute missing M instead?
weather_data.replace('M', 0, inplace=True)
weather_data.replace('T', 0, inplace=True)
weather_data.replace(' T', 0, inplace=True)
weather_data.replace('  T', 0, inplace=True)

In [110]:
#these have almost no data
#sunset/sunrise - not sure how these can contribute

weather_data.drop(columns=['Water1', 'SnowFall', 'Depth'], axis=1, inplace=True)

In [111]:
weather_data.head().T

,0,1,2,3,4
AvgSpeed,9.2,13.4,11.9,10.8,12.0
CodeSum,,BR,,RA,
Cool,2,0,0,0,0
Date,2007-05-01,2007-05-02,2007-05-03,2007-05-04,2007-05-05
Daytime,841,843,845,848,850
Depart,14,-3,2,4,5
DewPoint,51,42,40,41,38
Heat,0,14,9,7,5
PrecipTotal,0.00,0.00,0.00,0,0
ResultDir,27,4,7,8,7


In [112]:
#weather_data['Sunrise'].str[:2].astype(int)*60 + weather_data['Sunrise'].str[2:]

In [113]:
#changing dtypes for some variables that are viewed as strings

weather_data['SeaLevel'] = weather_data['SeaLevel'].astype(float)
weather_data['AvgSpeed'] = weather_data['AvgSpeed'].astype(float)
weather_data['PrecipTotal'] = weather_data['PrecipTotal'].astype(float)
weather_data['StnPressure'] = weather_data['StnPressure'].astype(float)
weather_data['WetBulb'] = weather_data['WetBulb'].astype(int)
weather_data['Depart'] = weather_data['Depart'].astype(int)

In [114]:
#converting some more variables from strings to integers. For some reason the above approach
#did not work for these.

new =[]
for i in weather_data['Tavg']:
    new.append(int(i))
weather_data['Tavg'] = new

new =[]
for i in weather_data['Heat']:
    new.append(int(i))
weather_data['Heat'] = new

new =[]
for i in weather_data['Cool']:
    new.append(int(i))
weather_data['Cool'] = new

In [115]:
#see if changing precip into fewer categories would make a difference

new_precip = []
for i in weather_data['PrecipTotal']:
    if i == 0:
        new_precip.append('No_Precip')
    if i>0 and i<=1:
        new_precip.append('Trace')
    if i>1:
        new_precip.append('Precip')
weather_data['PrecipCat'] = new_precip

In [116]:
# Alternative handling of these weather patterns:
# (patterns present vs not) EXCEPT all the missing values count as 0

patterns = ['SH', 'DZ', 'RA', 'TS', 'BR']
string = 'SH'  
count = []
for i in weather_data['CodeSum']:
    if any(x in i for x in patterns):
        count.append(0)
    else:
        count.append(1)
len(count)
weather_data['NewCodeSum'] = count
weather_data['NewCodeSum'].value_counts()

1    1651
0    1293
Name: NewCodeSum, dtype: int64

In [117]:
# we only care about Shower(SH), drizzle(DZ), rain(RA)
# Thunderstorm(TS), Mist(BR) = Each of these will reduce the number of mosquitoes

#I THINK THIS MAY BE ONLY PICKING OBSERVATIONS WHERE WE SEE ONE OF THESE, NOT COMBINED. ASK BRIAN IF THAT WAS
#THE INTENT (if we have SH DZ - we don't want that?)


patterns = ['SH', 'DZ', 'RA', 'TS', 'BR']

# iterate through all the CodeSum to find the columns with the patterns list
weather_data['CodeSum'] = weather_data['CodeSum'].apply(lambda x: [t for t in x.split('0') 
                                                                   if t in patterns])
# converts everything from the lists that were created in the previous code into strings
weather_data['CodeSum'] = weather_data['CodeSum'].apply(lambda x: 
                                                x if not isinstance(x, list) else x[0] if len(x) else 'None')

weather_data.CodeSum.value_counts()

None    2520
RA       296
BR       110
TS        10
DZ         8
Name: CodeSum, dtype: int64

In [118]:
weather_data.dtypes

AvgSpeed       float64
CodeSum         object
Cool             int64
Date            object
Daytime          int64
Depart           int64
DewPoint         int64
Heat             int64
PrecipTotal    float64
ResultDir        int64
ResultSpeed    float64
SeaLevel       float64
Station          int64
StnPressure    float64
Sunrise          int64
Sunrise_min      int64
Sunset           int64
Sunset_min       int64
Tavg             int64
Tmax             int64
Tmin             int64
WetBulb          int64
PrecipCat       object
NewCodeSum       int64
dtype: object

In [119]:
weather_data.head()

,AvgSpeed,CodeSum,Cool,Date,Daytime,Depart,DewPoint,Heat,PrecipTotal,ResultDir,...,Sunrise,Sunrise_min,Sunset,Sunset_min,Tavg,Tmax,Tmin,WetBulb,PrecipCat,NewCodeSum
0,9.2,None,2,2007-05-01,841,14,51,0,0.0,27,...,448,288,1849,1129,67,83,50,56,No_Precip,1
1,13.4,BR,0,2007-05-02,843,-3,42,14,0.0,4,...,447,287,1850,1130,51,59,42,47,No_Precip,0
2,11.9,None,0,2007-05-03,845,2,40,9,0.0,7,...,446,286,1851,1131,56,66,46,48,No_Precip,1
3,10.8,RA,0,2007-05-04,848,4,41,7,0.0,8,...,444,284,1852,1132,58,66,49,50,No_Precip,0
4,12.0,None,0,2007-05-05,850,5,38,5,0.0,7,...,443,283,1853,1133,60,66,53,49,No_Precip,1


In [120]:
#Heating degree days are summations of negative differences between the mean daily temperature
#and the 65°F base; cooling degree days are summations of positive differences from the same base.
#For example, cooling degree days for a station with daily mean temperatures during a seven-day 
#period of 67,65,70,74,78,65 and 68, are 2,0,5,9,13,0,and 3, for a total for the week of 32 cooling degree days.

#SO TAVG IS ACTUALLY SAME AS HEAT/COOL? Drop those.

new_cool = []
for i in weather_data['Cool']:
    new_cool.append(65+i)
len(new_cool)

2944

In [121]:
weather_data.drop(columns=['Heat','Cool'], axis=1, inplace=True)

In [122]:
#split weather by stations

station1 = weather_data[weather_data['Station']==1]
print(station1.shape)
station2 = weather_data[weather_data['Station']==2]
print(station2.shape)

(1472, 22)
(1472, 22)


In [124]:
#apply weather data from the stations closest to the data/test sets, merge on 'Date'

train_data_station1 = train_data_station1.merge(station1,on="Date")
train_data_station2 = train_data_station2.merge(station2,on="Date")
test_data_station1 = test_data_station1.merge(station1,on="Date")
test_data_station2 = test_data_station2.merge(station2,on="Date")

In [125]:
#combining test and train sets that belong to station 1 and 2

train_data = pd.concat([train_data_station1, train_data_station2])
test_data = pd.concat([test_data_station1, test_data_station2])

In [126]:
train_data.drop(columns=['Station_x','Station_y'], axis=1, inplace=True)
test_data.drop(columns=['Station_x','Station_y'], axis=1, inplace=True)

In [127]:
#confirming that the number of rows/observations is still the same
train_data.shape, test_data.shape

((10506, 32), (116293, 31))

In [128]:
#changing the order back to the original order
test_data = test_data.sort_values(by=['Id'])

In [129]:
test_data.tail(5)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,...,Sunrise,Sunrise_min,Sunset,Sunset_min,Tavg,Tmax,Tmin,WetBulb,PrecipCat,NewCodeSum
80887,116289,2014-10-02,"2100 North Cannon Drive, Chicago, IL 60614, USA",CULEX SALINARIUS,21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.63359,...,549,349,1732,1052,71,75,66,65,Trace,0
80888,116290,2014-10-02,"2100 North Cannon Drive, Chicago, IL 60614, USA",CULEX TERRITANS,21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.63359,...,549,349,1732,1052,71,75,66,65,Trace,0
80889,116291,2014-10-02,"2100 North Cannon Drive, Chicago, IL 60614, USA",CULEX TARSALIS,21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.63359,...,549,349,1732,1052,71,75,66,65,Trace,0
80890,116292,2014-10-02,"2100 North Cannon Drive, Chicago, IL 60614, USA",UNSPECIFIED CULEX,21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.63359,...,549,349,1732,1052,71,75,66,65,Trace,0
80891,116293,2014-10-02,"2100 North Cannon Drive, Chicago, IL 60614, USA",CULEX ERRATICUS,21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.63359,...,549,349,1732,1052,71,75,66,65,Trace,0


In [130]:
train_data.to_csv('train_final.csv', index = False)
test_data.to_csv('test_final.csv', index = False)